 ### Load the country Parquet file into a pandas DataFrame

In [1]:
import pandas as pd
import os
parquet_path = '../local_data/gold/gold_country_program_summary/part-00000-tid-438118432531490293-846e5ad2-ff6b-4689-bb8d-6456d6a7866a-4604-1-c000.snappy.parquet'

# Check if file exists first
assert os.path.exists(parquet_path), f"File not found: {parquet_path}"

# Load Parquet file into DataFrame
df_countries = pd.read_parquet(parquet_path, engine='pyarrow')
print(df_countries.head())


  institute_country  num_institutes  total_programs  avg_institute_ctr  \
0    United Kingdom               6             790              0.620   
1          Portugal               1              26              0.598   
2            Turkey               4             294              0.545   
3            Canada               4             320              0.526   
4     United States               3             128              0.521   

   total_views  total_impressions  country_rank  \
0       915932            2099792             1   
1         2852               5724             2   
2       311593             997145             3   
3       470027            1729844             4   
4       407046            1044612             5   

                                        top_programs  
0  [{'program_name': 'Zoology', 'ctr': 0.33578431...  
1  [{'program_name': 'Wine, Tourism and Innovatio...  
2  [{'program_name': 'Zootechnique', 'ctr': 0.852...  
3  [{'program_name': 'Workpl

### Preparing the data in the DataFrame for insert

In [2]:
#  converting the top_programs column to JSON strings:
import json
import numpy as np

def convert_for_json(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()  # convert ndarray to list
    raise TypeError(f"Type {type(obj)} not serializable")

df_countries['top_programs_json'] = df_countries['top_programs'].apply(
    lambda x: json.dumps(x, default=convert_for_json)
)

In [4]:
df_countries.columns

Index(['institute_country', 'num_institutes', 'total_programs',
       'avg_institute_ctr', 'total_views', 'total_impressions', 'country_rank',
       'top_programs', 'top_programs_json'],
      dtype='object')

In [5]:
# Rename column
df_countries = df_countries.rename(columns={'avg_institute_ctr': 'avg_ctr'})

# Keep only needed columns for DB insert
cols_to_keep = ['institute_country', 'num_institutes', 'total_programs', 'avg_ctr', 'total_views', 'total_impressions', 'top_programs_json']
df_countries_clean = df_countries[cols_to_keep]

# Rename 'top_programs_json' to 'top_programs' for DB column naming consistency
df_countries_clean = df_countries_clean.rename(columns={'top_programs_json': 'top_programs'})

print(df_countries_clean.head())

  institute_country  num_institutes  total_programs  avg_ctr  total_views  \
0    United Kingdom               6             790    0.620       915932   
1          Portugal               1              26    0.598         2852   
2            Turkey               4             294    0.545       311593   
3            Canada               4             320    0.526       470027   
4     United States               3             128    0.521       407046   

   total_impressions                                       top_programs  
0            2099792  [{"program_name": "Zoology", "ctr": 0.33578431...  
1               5724  [{"program_name": "Wine, Tourism and Innovatio...  
2             997145  [{"program_name": "Zootechnique", "ctr": 0.852...  
3            1729844  [{"program_name": "Workplace Safety and Preven...  
4            1044612  [{"program_name": "Visual and Performing Arts"...  


In [6]:
df_countries_clean.head()

,institute_country,num_institutes,total_programs,avg_ctr,total_views,total_impressions,top_programs
0,United Kingdom,6,790,0.620,915932,2099792,"[{""program_name"": ""Zoology"", ""ctr"": 0.33578431..."
1,Portugal,1,26,0.598,2852,5724,"[{""program_name"": ""Wine, Tourism and Innovatio..."
2,Turkey,4,294,0.545,311593,997145,"[{""program_name"": ""Zootechnique"", ""ctr"": 0.852..."
3,Canada,4,320,0.526,470027,1729844,"[{""program_name"": ""Workplace Safety and Preven..."
4,United States,3,128,0.521,407046,1044612,"[{""program_name"": ""Visual and Performing Arts""..."


In [2]:
import psycopg2
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

conn = psycopg2.connect(
    host=os.getenv("PG_HOST"),
    port=os.getenv("PG_PORT"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD"),
    dbname=os.getenv("PG_DB")
)

cur = conn.cursor()

In [8]:
import json

# Convert JSON string in 'top_programs_json' column into actual Python lists/dicts
df_countries['top_programs_parsed'] = df_countries['top_programs_json'].apply(json.loads)

# Then build records with the parsed JSON column, not the string column
records = list(df_countries[[
    'institute_country', 'num_institutes', 'total_programs', 'avg_ctr', 
    'total_views', 'total_impressions', 'top_programs_parsed'
]].itertuples(index=False, name=None))

In [14]:
conn.rollback()

In [9]:
import json

records = []
for row in df_countries.itertuples(index=False):
    top_programs_jsonb = json.loads(row.top_programs_json)
    records.append((
        row.institute_country,
        row.num_institutes,
        row.total_programs,
        row.avg_ctr,
        row.total_views,
        row.total_impressions,
        top_programs_jsonb
    ))

In [ ]:
from psycopg2.extras import execute_values, Json

# Create cursor
cur = conn.cursor()

# Prepare records as tuples, wrapping dict with Json()
records = [
    (
        row.institute_country,
        row.num_institutes,
        row.total_programs,
        row.avg_ctr,
        row.total_views,
        row.total_impressions,
        Json(row.top_programs_json)  
    )
    for row in df_countries.itertuples(index=False)
]

insert_query = """
INSERT INTO countries (
    institute_country, num_institutes, total_programs, avg_ctr, total_views, total_impressions, top_programs
)
VALUES %s
ON CONFLICT (institute_country) DO UPDATE SET
    num_institutes = EXCLUDED.num_institutes,
    total_programs = EXCLUDED.total_programs,
    avg_ctr = EXCLUDED.avg_ctr,
    total_views = EXCLUDED.total_views,
    total_impressions = EXCLUDED.total_impressions,
    top_programs = EXCLUDED.top_programs,
    updated_at = CURRENT_TIMESTAMP;
"""

execute_values(cur, insert_query, records)
conn.commit()

In [3]:
with conn.cursor() as cur:
    for table in ['countries', 'institutions', 'programs']:
        print(f"\nSchema for table {table}:")
        cur.execute(f"""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_name = '{table}';
        """)
        columns = cur.fetchall()
        for col in columns:
            print(f"{col[0]}: {col[1]}")


Schema for table countries:
updated_at: timestamp without time zone
country_id: integer
num_institutes: integer
total_programs: integer
avg_ctr: numeric
total_views: bigint
total_impressions: bigint
top_programs: jsonb
created_at: timestamp without time zone
institute_country: text

Schema for table institutions:
updated_at: timestamp without time zone
created_at: timestamp without time zone
institution_id: integer
country_id: integer
ctr: numeric
total_views: bigint
total_impressions: bigint
institution_name: text
institution_type: text

Schema for table programs:
updated_at: timestamp without time zone
program_id: integer
institution_id: integer
country_id: integer
duration_months: integer
tuition: numeric
ctr: numeric
total_views: bigint
total_impressions: bigint
created_at: timestamp without time zone
program_name: text
